# Homework - experiment tracking

In [1]:
import mlflow

## Installing the package

In [2]:
# !pipenv install mlflow
!mlflow --version

mlflow, version 2.3.2


## Download and preprocess the data

In [3]:
#!python preprocess_data.py --raw_data_path ../data --dest_path ./output
!du -h ./output/dv.pkl 

152K	./output/dv.pkl


## Train a model with autolog

In [4]:
# check ./train.py file
!python train.py

2023/06/01 08:57:06 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2023/06/01 08:57:06 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Running upgrade  -> 451aebb31d03, add metric step
INFO  [alembic.runtime.migration] Running upgrade 451aebb31d03 -> 90e64c465722, migrate user column to tags
INFO  [alembic.runtime.migration] Running upgrade 90e64c465722 -> 181f10493468, allow nulls for metric values
INFO  [alembic.runtime.migration] Running upgrade 181f10493468 -> df50e92ffc5e, Add Experiment Tags Table
INFO  [alembic.runtime.migration] Running upgrade df50e92ffc5e -> 7ac759974ad8, Update run tags with larger limit
INFO  [alembic.runtime.migration] Running upgrade 7ac759974ad8 -> 89d4b8295536, create latest metrics table
INFO  [89d4b8295536_create_latest_metrics_table_py] Migration complete!
INFO  

In [5]:
from IPython.display import display, HTML
display(HTML(data="./mlruns/1/ed4405a08db548e986f0960847d22c3f/artifacts/estimator.html"))

## Tune model hyperparameters

In [6]:
# check ./hpo.py file
!python hpo.py

2023/06/01 08:59:56 INFO mlflow.tracking.fluent: Experiment with name 'random-forest-hyperopt' does not exist. Creating a new experiment.
[I 2023-06-01 08:59:56,843] A new study created in memory with name: no-name-9ca3f363-37dc-4102-8608-0ba3fce15231
[I 2023-06-01 08:59:58,293] Trial 0 finished with value: 2.451379690825458 and parameters: {'n_estimators': 25, 'max_depth': 20, 'min_samples_split': 8, 'min_samples_leaf': 3}. Best is trial 0 with value: 2.451379690825458.
[I 2023-06-01 08:59:58,512] Trial 1 finished with value: 2.4667366020368333 and parameters: {'n_estimators': 16, 'max_depth': 4, 'min_samples_split': 2, 'min_samples_leaf': 4}. Best is trial 0 with value: 2.451379690825458.
[I 2023-06-01 08:59:59,869] Trial 2 finished with value: 2.449827329704216 and parameters: {'n_estimators': 34, 'max_depth': 15, 'min_samples_split': 2, 'min_samples_leaf': 4}. Best is trial 2 with value: 2.449827329704216.
[I 2023-06-01 09:00:00,456] Trial 3 finished with value: 2.460983516558473 a

## Promote the best model to the model registry

In [7]:
# check register_model.py file
!python register_model.py

2023/06/01 09:05:49 INFO mlflow.tracking.fluent: Experiment with name 'random-forest-best-models' does not exist. Creating a new experiment.
2023/06/01 09:05:52 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/rafaelsguerra/.local/share/virtualenvs/MLOps-zoomcamp-jgOQhffP/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils."
Successfully registered model 'nyc-green-taxy-regressor-RF'.
2023/06/01 09:06:02 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: nyc-green-taxy-regressor-RF, version 1
Created version '1' of model 'nyc-green-taxy-regressor-RF'.


In [8]:
mlflow.set_tracking_uri("sqlite:///mlflow.db")
client = mlflow.MlflowClient()
model = client.get_registered_model("nyc-green-taxy-regressor-RF")
print(model)

<RegisteredModel: aliases={}, creation_timestamp=1685621162549, description=None, last_updated_timestamp=1685621162561, latest_versions=[<ModelVersion: aliases=[], creation_timestamp=1685621162561, current_stage='None', description=None, last_updated_timestamp=1685621162561, name='nyc-green-taxy-regressor-RF', run_id='6b6c0cb28fbb4dae8e1795d7d3a0a538', run_link=None, source=('/home/rafaelsguerra/Documentos/MLOps-zoomcamp/02 - experiment '
 'tracking/mlruns/3/6b6c0cb28fbb4dae8e1795d7d3a0a538/artifacts/model'), status='READY', status_message=None, tags={}, user_id=None, version=1>], name='nyc-green-taxy-regressor-RF', tags={}>


In [9]:
run = mlflow.get_run('6b6c0cb28fbb4dae8e1795d7d3a0a538')
test_rmse = run.data.metrics["test_rmse"]
print(f"Test RMSE: {test_rmse:.3f}")

Test RMSE: 2.285
